# Homework 7 solutions

## 1. Electron heat capacity

We want to find the specific heat of degenerate non-relativistic electrons and compare to that of the ions in a white dwarf to see if we were justified in neglecting their contribution when we did white dwarf cooling.

In [1]:
import numpy as np

We'll use `unyt` to keep track of units for us.  This uses SymPy underneat to symbolically track the units we attach to quantities.

In [9]:
import unyt as u

In a previous homework, we found:

\begin{equation}    
    P_e = \frac{h^2}{20m_e} \left ( \frac{3}{\pi} \right )^{2/3} n_e^{5/3}               
        \left [ 1 + \frac{40\pi^2}{(3/\pi)^{4/3}} \frac{m_e^2}{h^4} \frac{(kT)^2}{n_e^{4/3}}
        \right ]             
\end{equation} 
  
We'll write this as:

$$P_e = \alpha n_e^{5/3} \left [ 1 + \beta \frac{T^2}{n_e^{4/3}} \right ]$$

with

$$\alpha =  \frac{h^2}{20m_e} \left ( \frac{3}{\pi} \right )^{2/3}$$

and 

$$\beta = \frac{40\pi^2}{(3/\pi)^{4/3}} \frac{m_e^2}{h^4} k^2$$

Let's compute these

In [10]:
h = 6.63e-27 * u.erg * u.s
m_u = 1.67e-24 * u.g
m_e = 9.11e-28 * u.g
k = 1.38e-16 * u.erg / u.K

In [11]:
alpha = h**2 / (20 * m_e) * (3 / np.pi)**(2./3.)
alpha

unyt_quantity(2.33951736e-27, 'erg**2*s**2/g')

In [12]:
beta = 40 * np.pi**2 / (3 / np.pi)**(4./3.) * m_e**2 / h**4 * k**2
beta

unyt_quantity(3.43404019e+21, 'g**2/(K**2*erg**2*s**4)')

Now, we know that for non-relativistic electrons,

$$\rho e_e = \frac{3}{2} P_e$$

and

$$n_e = \frac{\rho}{\mu_e m_u}$$

so

$$e_e = \frac{3}{2} \alpha \frac{n_e^{5/3}}{\rho} \left [ 1 + \beta \frac{T^2}{n_e^{4/3}} \right ]$$

and

$$c_v = \left . \frac{\partial e_e}{\partial T} \right |_\rho = 3 \alpha \beta \frac{T}{\rho^{2/3}(\mu_e m_u)^{4/3}}$$

If we take typical conditions in a white dwarf, we can evaluate this

In [13]:
rho = 1.e7 * u.g / u.cm**3
T = 1.e7 * u.K
mu_e = 0.5

In [14]:
cv_e = 3 * alpha * beta * T / rho**(2./3.) / (mu_e * m_u)**(1./3.)
cv_e

unyt_quantity(551430.44751741, 'cm**2/(K*s**2)')

In [15]:
cv_e.to("erg/g/K")

unyt_quantity(551430.44751741, 'erg/(K*g)')

We see that for the electrons,

$$c_{v,e} = 5.5\times 10^5~\mathrm{erg / g / K}$$

For the ions, in class we found that

$$c_v = \frac{3}{2} \frac{k}{\mu_I m_u}$$

we can evaluate this for a carbon white dwarf ($\mu_I = 12$):

In [17]:
mu_I = 12
cv_ions = 1.5 * k / (mu_I * m_u)
cv_ions

unyt_quantity(10329341.31736527, 'erg/(K*g)')

We see that we have

$$c_{v,ions} = 10^7~\mathrm{erg/g/K}$$

so $c_{v,ions} \gg c_{v,e}$, justifying our assumption in class.

## 2. WD cooling

Another assumption we made when we did white dwarf cooling was to assume that the radiative layer was thin.  Here we'll check that.

We showed that the radiative layer behaves like a polytrope with an equation of state:

$$P = K^\prime T^{1+n_\mathrm{eff}}$$

with $n_\mathrm{eff} = 13/4$.

Let's start with hydrostatic equilibrium:

$$\frac{dP}{dr} = -\rho \frac{G M_\star}{r^2}$$

where $M_\star$ is assumed to be the constant mass of the star (the layer does not have
significant masss).

Using the chain rule, we can write:

$$\frac{dP}{dr} = \frac{dP}{dT} \frac{dT}{dr} = \frac{1}{\nabla} \frac{P}{T} \frac{dT}{dr}$$

and from our EOS, we see $\nabla^{-1} = 1 + n_\mathrm{eff} =  17/4$.

Now we use the ideal gas law to replace $P/T = \rho k /(\mu m_u)$, giving

$$\frac{dT}{dr} = -\frac{\mu m_u}{k}  \nabla \frac{GM_\star}{r^2}$$

We can integrate this from the transition radius (where $T = T_c$) to the surface of the star

$$\int_{T_c}^{T_\star} dT = - \frac{\mu m_u}{k} \nabla G M_\star \int_{r_\mathrm{tr}}^{R_\star} \frac{1}{{r^\prime}^2} dr^\prime$$

which gives

$$T_\star - T_c = \nabla \frac{\mu m_u}{k} G M_\star \left ( \frac{1}{r_\mathrm{tr}} - \frac{1}{R_\star} \right )$$

we can take the surface temperature to be 0, giving

$$\frac{k T_c}{\mu m_u} = \nabla \frac{GM_\star}{R_\star} \frac{R_\star - r_\mathrm{tr}}{r_\mathrm{tr}} \sim \nabla g l$$

with $l = R_\star - r_\mathrm{tr}$ as the layer thickness.

This gives

$$l \sim \frac{17}{4} \frac{k T_c}{\mu m_u g}$$

Let's put in numbers

In [34]:
M = 0.6 * u.solar_mass_cgs
R = 0.012 * 7.e10 * u.cm
G = 6.67e-8 * u.dyne * u.cm**2 / u.g**2

In [38]:
g = G * M / R**2
g.to("cm/s**2")

unyt_quantity(1.12778349e+08, 'cm/s**2')

In [40]:
Tc = 1.e8 * u.K
mu = 12

In [52]:
l = 17./4. * k * Tc / (mu * m_u * g)
(l/R).to("dimensionless")                      

unyt_quantity(0.03089337, 'dimensionless')

We see that for $T_c = 10^7~\mathrm{K}$ the layer is indeed thin.

We also see that if we increase $T_c$ by a factor of 10, then $l \sim R_\star$, so our approximation is no longer as good.